In [1]:
import pandas as pd
import nltk 
from nltk.metrics.distance  import edit_distance, jaccard_distance # edit distance is the same as Levenshtein edit-distance
import stanza
import re

In [2]:
# use the stanza tokenizer
stanza.download('sv', processors='tokenize,pos,lemma,depparse')
nlp = stanza.Pipeline(lang='sv', processors='tokenize,pos,lemma,depparse')

2022-04-08 09:52:56 INFO: Downloading these customized packages for language: sv (Swedish)...
| Processor | Package   |
-------------------------
| tokenize  | talbanken |
| pos       | talbanken |
| lemma     | talbanken |
| depparse  | talbanken |
| pretrain  | talbanken |

2022-04-08 09:52:56 INFO: File exists: /Users/filippakarrfelt/stanza_resources/sv/tokenize/talbanken.pt.
2022-04-08 09:52:56 INFO: File exists: /Users/filippakarrfelt/stanza_resources/sv/pos/talbanken.pt.
2022-04-08 09:52:56 INFO: File exists: /Users/filippakarrfelt/stanza_resources/sv/lemma/talbanken.pt.
2022-04-08 09:52:57 INFO: File exists: /Users/filippakarrfelt/stanza_resources/sv/depparse/talbanken.pt.
2022-04-08 09:52:57 INFO: File exists: /Users/filippakarrfelt/stanza_resources/sv/pretrain/talbanken.pt.
2022-04-08 09:52:57 INFO: Finished downloading models and saved to /Users/filippakarrfelt/stanza_resources.
2022-04-08 09:52:57 INFO: Loading these models for language: sv (Swedish):
| Processor | Package  

In [3]:
# https://github.com/almgru/svenska-ord.txt
# add all the words from the corpora
def read_corpus(filename):
    file = open(filename, 'r')
    lines = file.readlines()
    
    corpus = set()
    count = 0
    # Strips the newline character
    for word in lines:
        count += 1
        corpus.add(word.strip())
    print("Added {} words to corpus.".format(count))
    return corpus

In [4]:
corpus = read_corpus('../svenska-ord.txt')
corpus_list = list(corpus)

context_corpus = read_corpus('../context-corpus.txt')
context_corpus_list = list(context_corpus)

Added 122314 words to corpus.
Added 29358 words to corpus.


In [18]:
# load the dataset
df_train = pd.read_pickle("./data_frames/parsed_json_data/df_train_original.pkl")
df_train.head()

,choices,context,question
0,"[{'end': 224, 'extra': None, 'start': 175, 'te...",Visum för affärs- och konferensbesök\r\nOm du ...,Vad är ett visum?
1,"[{'end': 707, 'extra': None, 'start': 681, 'te...",Eget företag\r\nEfter beslut\r\nBeslutet skick...,Vad ska du ta med när du hämtar ditt beslut?
2,"[{'end': 1165, 'extra': None, 'start': 1154, '...",Utbytesdoktorander\r\nIntervju\r\nDu kommer at...,Hur länge gäller ett uppehållstillstånd för be...
3,"[{'end': 598, 'extra': None, 'start': 589, 'te...",Eget företag\r\nNär vi har tagit emot din ansö...,Vad är ett uppehållstillståndskort?
4,"[{'end': 1932, 'extra': None, 'start': 1924, '...",Flytta tillbaka till Sverige\r\nSå ansöker du\...,Vad är ett uppehållstillståndskort?


In [9]:
def replace_words(doc, useBigCorpus=True): 
    all_sentences = []
    replaced_words = []
    for sentence in doc.sentences:
        current_sentence = []
        for raw_word in sentence.words:
            # only add if character is letter or number (removes , . ? ! etc.)
            word = raw_word.text
            lemma = raw_word.lemma
            word = word.lower()
            word  = re.sub('[^\sa-zåäöA-ZÅÄÖ0-9-]', '', word)
            if len(word) > 0 and not word.isnumeric():
                temp = None
                if useBigCorpus:
                    if not word in corpus and not lemma in corpus:
                        # print('word not in corpus! ', word)
                        temp = [(edit_distance(word, w),w) for w in corpus_list]
                else:
                    if not word in context_corpus and not lemma in context_corpus:
                        if not word in corpus and not lemma in corpus:
                            temp = [(edit_distance(word, w),w) for w in context_corpus_list]
                if temp:
                    suggestion = sorted(temp, key = lambda val:val[0])[0][1]
                    suggestion_score = sorted(temp, key = lambda val:val[0])[0][0]
                    print('word: ', word)
                    print('suggestion: ', suggestion)
                    print('suggestion score: ', suggestion_score)
                    if suggestion_score < 3: # a maximum of 2 operations needed for the best option..
                        current_sentence.append(suggestion)
                        replaced_words.append((word, suggestion))
                    else:
                        current_sentence.append(raw_word.text)
                else:
                    current_sentence.append(raw_word.text)
            else:
                current_sentence.append(raw_word.text)
        all_sentences.append(current_sentence)
    return all_sentences[0], replaced_words

def question_spell_correction(df, useBigCorpus=True):
    print('correcting question words..')
    parsed_questions = []
    all_replaced_words = []
    for index, row in df.iterrows():
        if index % 100 == 0:
            print('doc nr {}'.format(index))
        question = row['question']
        question_parsed = nlp(question)
        # TODO: also do this for the correct answer..
        parsed_question, replaced_words = replace_words(question_parsed, useBigCorpus)
        parsed_questions.append(parsed_question)
        all_replaced_words.append(replaced_words)
    return parsed_questions, all_replaced_words

In [10]:
# save context corpus to file
def save_corrected_words(filename, list):
    list.sort()
    with open(filename, 'w') as out:
        for word_arr in list:
            if len(word_arr) > 0:
                for word in word_arr:
                    out.write(word[0] + ' - ' + word[1] + '\n')


In [11]:
parsed_questions_big, replaced_words_big = question_spell_correction(df_train, True) # use big corpus
df_train['question_spell_checked_big_corpus'] = parsed_questions_big
save_corrected_words('./figures/spell_check/spell_check_big_corpus.txt', replaced_words_big)
df_train.head()

correcting question words..
doc nr 0
word:  uppehållstillståndskort
suggestion:  uppehållstillstånd
suggestion score:  5


KeyboardInterrupt: 

In [139]:
parsed_questions_context_corpus, replaced_words_context_corpus = question_spell_correction(df_train, False) # use context corpus
df_train['question_spell_checked_context_corpus'] = parsed_questions_context_corpus
save_corrected_words('./figures/spell_check/spell_check_context_corpus.txt', replaced_words_context_corpus)

correcting question words..
doc nr 0
word:  tillsynsinstsen
suggestion:  tillsynsinsats
suggestion score:  3
word:  felsatt
suggestion:  nedsatt
suggestion score:  2
word:  vemvilka
suggestion:  vilka
suggestion score:  3
word:  tillträdande
suggestion:  tillträdde
suggestion score:  2
word:  viket
suggestion:  vikt
suggestion score:  1
word:  udvika
suggestion:  undvika
suggestion score:  1
doc nr 100
word:  parkingsvakt
suggestion:  parkeringsvakt
suggestion score:  2
word:  socialförsäkringshandläggarna
suggestion:  socialförsäkringshandläggare
suggestion score:  2
word:  medskickande
suggestion:  medsökande
suggestion score:  3
word:  kontoutraget
suggestion:  kontoutdraget
suggestion score:  1
word:  tillheltidsarbete
suggestion:  heltidsarbete
suggestion score:  4
doc nr 200
word:  äldra
suggestion:  ändra
suggestion score:  1
word:  skogägarföreningarna
suggestion:  skogsägarföreningarna
suggestion score:  1
word:  indrottstränare
suggestion:  idrottstränare
suggestion score:  1

,choices,context,question,question_spell_checked_context_corpus
0,"[{'end': 224, 'extra': None, 'start': 175, 'te...",Visum för affärs- och konferensbesök\r\nOm du ...,Vad är ett visum?,"[Vad, är, ett, visum, ?]"
1,"[{'end': 707, 'extra': None, 'start': 681, 'te...",Eget företag\r\nEfter beslut\r\nBeslutet skick...,Vad ska du ta med när du hämtar ditt beslut?,"[Vad, ska, du, ta, med, när, du, hämtar, ditt,..."
2,"[{'end': 1165, 'extra': None, 'start': 1154, '...",Utbytesdoktorander\r\nIntervju\r\nDu kommer at...,Hur länge gäller ett uppehållstillstånd för be...,"[Hur, länge, gäller, ett, uppehållstillstånd, ..."
3,"[{'end': 598, 'extra': None, 'start': 589, 'te...",Eget företag\r\nNär vi har tagit emot din ansö...,Vad är ett uppehållstillståndskort?,"[Vad, är, ett, uppehållstillståndskort, ?]"
4,"[{'end': 1932, 'extra': None, 'start': 1924, '...",Flytta tillbaka till Sverige\r\nSå ansöker du\...,Vad är ett uppehållstillståndskort?,"[Vad, är, ett, uppehållstillståndskort, ?]"


In [ ]:
# save spell checked df
df_train.to_pickle("./data_frames/auto_spell_check/df_train_auto_spell_check.pkl")